In [ ]:
#从google云盘上加载数据
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
os.chdir('/content/drive/MyDrive/transformers/天池-入门NLP - 新闻文本分类')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
import numpy as np
from scipy import sparse

"""
df_train = pd.read_csv('./train_set.csv',sep='\t')
df_test = pd.read_csv('./test_a.csv',sep='\t')

df_train['text_len'] = df_train['text'].apply(lambda x:len(x.split(' ')))
"""
print(df_train['text_len'].describe())

# 词频5000
tfidfVector=TfidfVectorizer(ngram_range=(1,3),max_features=5000)
tfidfVector.fit(pd.concat([df_train['text'],df_test['text']],axis=0))
#
X_train=tfidfVector.transform(df_train['text'])
X_test=tfidfVector.transform(df_test['text'])
#
sparse.save_npz('X_train_tfidf.npz',X_train)
sparse.save_npz('X_test_tfidf.npz',X_test)

count    200000.000000
mean        907.207110
std         996.029036
min           2.000000
25%         374.000000
50%         676.000000
75%        1131.000000
max       57921.000000
Name: text_len, dtype: float64


In [ ]:
#崩溃后重新加载数据
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from lightgbm import LGBMClassifier
import numpy as np
from scipy import sparse
X_train = sparse.load_npz('X_train_tfidf.npz')
X_test = sparse.load_npz('X_test_tfidf.npz')
df_train = pd.read_csv('./train_set.csv',sep='\t')

In [13]:
#clf1=RidgeClassifier()
#clf2=LogisticRegression()
#clf3=RandomForestClassifier

params_sklearn = {
    'learning_rate':0.1,
    'max_bin':150,
    'num_leaves':32,    
    'max_depth':11,
    'reg_alpha':0.1,
    'reg_lambda':0.2,   
    'objective':'multiclass',
    'n_estimators':100,
    #'class_weight':weight
}

clf=LGBMClassifier(**params_sklearn)
"""
clf=LGBMClassifier(n_jobs=-1,min_child_samples=21,max_depth=-1,subsample=0.7217, 
colsample_bytree=0.6,reg_alpha=0.001,reg_lambda=0.5,num_leaves=67,
learning_rate=0.088,n_estimators=100)
"""

clf.fit(X_train.todense(),df_train['label'].values)

df=pd.DataFrame()
df['label']=clf.predict(X_test)
df.to_csv('./TFIDF_lgb.csv')

KeyboardInterrupt: ignored

In [8]:
test=pd.read_csv('/TFIDF_lgb.csv')
test

,Unnamed: 0,label
0,0,1
1,1,2
2,2,8
3,3,5
4,4,0
...,...,...
49995,49995,0
49996,49996,13
49997,49997,1
49998,49998,3


In [12]:
test.to_csv('./TF-IDF_lgb.csv')

In [ ]:
# Count Vectors + RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=15000)

vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.74

0.7423139613717681


In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df=pd.read_csv('./train_set.csv',sep='\t',nrows=3000)

tfidf=TfidfVectorizer(ngram_range=(1,3),max_features=4000,stop_words=['3750','648','900'])
train_data=tfidf.fit_transform(train_df['text'][:2000]).todense())

clf=RidgeClassifier()
clf.fit(train_data,train_df['label'][2000:].values)
val_pred = clf.predict(train_data)
print(f1_score(train_df['label'][2000:].values,val_pred,average='macro'))

0.975062464835336


In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=3000)
# 将一篇文章映射为3000维度的向量
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=4000,stop_words=['3750','648','900'])
train_test = tfidf.fit_transform(train_df['text']).todense()

clf = RidgeClassifier()
clf.fit(train_test[:2000], train_df['label'].values[:2000])

val_pred = clf.predict(train_test[2000:])
print(f1_score(train_df['label'].values[2000:], val_pred, average='macro'))


0.8352693574249846


In [ ]:
#test_df=pd.read_csv('./test_a.csv',sep='\t')
#test_data=tfidf.fit_transform(test_df['text'])
#test_pred=clf.predict(test_data.todense())
#test_pred


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb
from sklearn.metrics import f1_score
train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=30000)
# 将一篇文章映射为3000维度的向量
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])
params_sklearn = {
    'learning_rate':0.1,
    'max_bin':150,
    'num_leaves':32,    
    'max_depth':11,
    'reg_alpha':0.1,
    'reg_lambda':0.2,   
    'objective':'multiclass',
    'n_estimators':300,
    #'class_weight':weight
}

clf = lgb.LGBMClassifier(**params_sklearn)
clf.fit(train_test[:20000].todense(), train_df['label'].values[:20000])

val_pred = clf.predict(train_test[20000:].todense())
print(f1_score(train_df['label'].values[20000:], val_pred, average='macro'))
#0.8923318050136503


0.9027526863541893


In [ ]:
# TF-IDF+高斯朴素贝叶斯

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=30000)
# 将一篇文章映射为3000维度的向量
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])
clf = GaussianNB()
clf.fit(train_test[:20000].toarray(), train_df['label'].values[:20000])

val_pred = clf.predict(train_test[20000:].toarray())
print(f1_score(train_df['label'].values[20000:], val_pred, average='macro'))

0.7286027535218247


In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=30000)
# 将一篇文章映射为3000维度的向量
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])
clf = DecisionTreeClassifier(criterion="entropy")
clf.fit(train_test[:20000].todense(), train_df['label'].values[:20000])

val_pred = clf.predict(train_test[20000:].todense())
print(f1_score(train_df['label'].values[20000:], val_pred, average='macro'))


0.7019031297922049


In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=30000)
# 将一篇文章映射为3000维度的向量
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])
clf = RandomForestClassifier(
        n_estimators=10, criterion='gini',
        max_depth=None,min_samples_split=2, 
        min_samples_leaf=1, min_weight_fraction_leaf=0.0,
        max_features='auto', max_leaf_nodes=None,
        min_impurity_split=1e-07,bootstrap=True,
        oob_score=False, n_jobs=1, 
        random_state=None, verbose=0,
        warm_start=False, class_weight=None)
clf.fit(train_test[:20000].todense(), train_df['label'].values[:20000])

val_pred = clf.predict(train_test[20000:].todense())
print(f1_score(train_df['label'].values[20000:], val_pred, average='macro'))

/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:301: FutureWarning: The min_impurity_split parameter is depr

0.7456517053768087


In [ ]:
#TF-IDF+XGBoost
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score
train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=30000)
# 将一篇文章映射为3000维度的向量
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])
clf = XGBClassifier(min_child_weight=6,max_depth=15,
                            objective='multi:softmax',num_class=5)
clf.fit(train_test[:20000].todense(), train_df['label'].values[:20000])

val_pred = clf.predict(train_test[20000:].todense())
print(f1_score(train_df['label'].values[20000:], val_pred, average='macro'))

0.9024422794080935
